# CSV to Google Sheets Uploader

This notebook reads CSV data and uploads it to Google Sheets using the same endpoint and data structure as your existing Node.js implementation.

## Features:
- Reads CSV files using pandas
- Transforms data to match your User model structure
- Handles date formatting
- Uploads to Google Sheets via Apps Script endpoint
- Comprehensive error handling and logging

In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.0 MB 9.3 MB/s eta 0:00:02
   ----------- ---------------------------- 3.1/11.0 MB 11.6 MB/s eta 0:00:01
   ------------------- -------------------- 5.2/11.0 MB 12.2 MB/s eta 0:00:01
   --------------------- ------------------ 6.0/11.0 MB 9.1 MB/s eta 0:00:01
   ------------------------ --------------- 6.8/11.0 MB 7.7 MB/s eta 0:00:01
   -------------------------- ------------- 7.3/11.0 MB 7.1 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.0 MB 6.1 MB/s eta 0:00:01
   ----------------------------- ---------- 8.1/11.0 MB 5.4 MB/s eta 0:00:01
   ------------------------------ --------- 8.4/11.0 MB 4.8 MB/s eta 0:00:01
   ---------------------

In [2]:
# Import required libraries
import pandas as pd
import requests
import json
from datetime import datetime
import logging
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [9]:
# Configuration - Update these values as needed
SHEET_URL = "https://script.google.com/macros/s/AKfycbxM_OfnwX9wo3DqlIV_G5C0pD_InJSvSa7UcxqERazPjoldIooZDO9SfMJTUvxCssq7/exec"
SHEET_ID = "1ztK5yqeGP83c4db7kbWsi7jQvyQhug7fJb-UtrpBdWM"

# CSV file path - Update this to your CSV file location
CSV_FILE_PATH = "pnb-metlife.users.csv"  # Replace with your actual CSV file path

print(f"📊 Target Sheet ID: {SHEET_ID}")
print(f"📁 CSV File: {CSV_FILE_PATH}")

📊 Target Sheet ID: 1ztK5yqeGP83c4db7kbWsi7jQvyQhug7fJb-UtrpBdWM
📁 CSV File: pnb-metlife.users.csv


In [10]:

def process_dataframe(df: pd.DataFrame) -> List[List[str]]:
    """Process DataFrame and return formatted values for sheets"""
    # Define the expected column order based on your User model
    expected_columns = [
        '_id', 'name', 'mobile', 'consent', 'channel', 'employeeCode', 
        'buisnessCode', 'city', 'age', 'monthlyExpense', 'retirementAge', 
        'futureValue', 'retirementCorpus', 'yearlyInvestment', 'createdAt', 
        'updatedAt', 'utm', 'utm_source', 'utm_campaign'
    ]
    
    # Create a new DataFrame with the expected structure
    processed_df = pd.DataFrame()
    
    values = processed_df.values.tolist()
    
    return values

In [28]:
class GoogleSheetsClient:
    """Client for interacting with Google Sheets via Apps Script endpoint"""
    
    def __init__(self, sheet_url: str, sheet_id: str):
        self.sheet_url = sheet_url
        self.sheet_id = sheet_id
        self.session = requests.Session()  # <-- create a session
    
    def upload_data(self, data: List[List[str]]) -> bool:
        """Upload data to Google Sheets"""
        try:
            payload = {
                "mode": "append",
                "data": data,
                "sheetId": self.sheet_id
            }
            
            logger.info(f"Uploading {len(data)} rows to Google Sheets")
            logger.info(f"Payload preview: {payload['data'][:2] if len(payload['data']) > 0 else 'No data'}")
            
            response = self.session.post(
                self.sheet_url,
                json=payload,
                timeout=30
            )
            
            response.raise_for_status()
            
            logger.info("✅ Data successfully uploaded to Google Sheets")
            logger.info(f"Response: {response.text}")
            return True
            
        except requests.exceptions.RequestException as e:
            logger.error(f"❌ Failed to upload data: {e}")
            return False
        except Exception as e:
            logger.error(f"❌ Unexpected error during upload: {e}")
            return False

# Initialize the client
sheets_client = GoogleSheetsClient(SHEET_URL, SHEET_ID)

In [57]:
# Read and process CSV data
try:
    # Read CSV file
    logger.info(f"📖 Reading CSV file: {CSV_FILE_PATH}")
    df = pd.read_csv(CSV_FILE_PATH)
    
    df.drop(columns=['__v'], inplace=True)
    # Display basic info about the data
    print(f"📊 CSV loaded successfully!")
    print(f"📈 Total rows: {len(df)}")
    print(f"🏷️  Columns: {list(df.columns)}")
    print("\n🔍 First few rows:")

    
    # display(df.head())
    # Process the data
    logger.info("🔄 Processing data for Google Sheets...")

    df = df.fillna("") 

    df = df.reindex(columns=[
        '_id', 'name', 'mobile', 'consent', 'channel', 'employeeCode', 
        'buisnessCode', 'city', 'age', 'monthlyExpense', 'retirementAge', 
        'futureValue', 'retirementCorpus', 'yearlyInvestment', 'createdAt', 
        'updatedAt', 'utm', 'utm_source', 'utm_campaign'
    ])

    if 'mobile' in df.columns:
        df['mobile'] = pd.to_numeric(df['mobile'], errors='coerce').fillna(0).astype(int)

    processed_data = df.values.tolist()

    print(processed_data[0])
    
    print(f"✅ Data processed successfully!")
    print(f"📤 Ready to upload {len(processed_data)} rows")
    
except FileNotFoundError:
    print(f"❌ CSV file not found: {CSV_FILE_PATH}")
    print("Please update the CSV_FILE_PATH variable with the correct path to your CSV file.")
except Exception as e:
    print(f"❌ Error reading CSV: {e}")



BATCH_SIZE = 500

def chunked_upload(client, data, batch_size=BATCH_SIZE):
    total = len(data)
    for start in range(0, total, batch_size):
        end = start + batch_size
        batch = data[start:end]
        print(f"📤 Uploading rows {start}–{end-1} ({len(batch)} rows)")
        
        success = client.upload_data(batch)
        if not success:
            print(f"❌ Failed to upload batch {start}–{end-1}")
            break


# processed_data = df.values.tolist()  # already prepared
chunked_upload(sheets_client, processed_data, batch_size=1000)

2025-08-22 04:39:44,748 - INFO - 📖 Reading CSV file: pnb-metlife.users.csv
2025-08-22 04:39:44,756 - INFO - 🔄 Processing data for Google Sheets...
2025-08-22 04:39:44,761 - INFO - Uploading 542 rows to Google Sheets
2025-08-22 04:39:44,762 - INFO - Payload preview: [['68a7066dc3e5abf79d0452ae', 'Zeeshan Mohammad', 7899766044, True, '', 56.0, 'Bengaluru', '', 'fire_aug_25 ', '2025-08-21T11:43:41.116Z', '2025-08-21T11:43:47.009Z', 0, 31.0, 60000.0, 'social_media', '', 3090146.92, 104323334.32, 1644449.23], ['68a7069dc3e5abf79d0452b3', 'Raj', 7899768250, True, '', 57.0, 'Bangalore', '', 'fire_aug_25 ', '2025-08-21T11:44:29.167Z', '2025-08-21T11:44:34.580Z', 0, 34.0, 60000.0, 'social_media', '', 2750219.76, 84997336.22, 1524623.69]]


📊 CSV loaded successfully!
📈 Total rows: 542
🏷️  Columns: ['_id', 'name', 'mobile', 'consent', 'channel', 'city', 'createdAt', 'updatedAt', 'age', 'monthlyExpense', 'retirementAge', 'futureValue', 'retirementCorpus', 'yearlyInvestment', 'employeeCode', 'utm', 'utm_source', 'utm_campaign', 'buisnessCode']

🔍 First few rows:
['68a7066dc3e5abf79d0452ae', 'Zeeshan Mohammad', 7899766044, True, '', 56.0, 'Bengaluru', '', 'fire_aug_25 ', '2025-08-21T11:43:41.116Z', '2025-08-21T11:43:47.009Z', 0, 31.0, 60000.0, 'social_media', '', 3090146.92, 104323334.32, 1644449.23]
✅ Data processed successfully!
📤 Ready to upload 542 rows
📤 Uploading rows 0–999 (542 rows)


2025-08-22 04:40:15,199 - ERROR - ❌ Failed to upload data: HTTPSConnectionPool(host='script.google.com', port=443): Read timed out. (read timeout=30)


❌ Failed to upload batch 0–999


In [32]:
print(len(processed_data))

9826


In [33]:
# Upload data to Google Sheets
if 'processed_data' in locals() and processed_data:
    logger.info("🚀 Starting upload to Google Sheets...")
    
    success = sheets_client.upload_data(processed_data)
    
    if success:
        print("🎉 Upload completed successfully!")
        print(f"📊 {len(processed_data)} rows uploaded to Google Sheets")
    else:
        print("💥 Upload failed! Check the logs above for details.", success)
else:
    print("⚠️  No processed data available. Please run the previous cell first.")

2025-08-21 19:03:02,256 - INFO - 🚀 Starting upload to Google Sheets...
2025-08-21 19:03:02,257 - INFO - Uploading 9826 rows to Google Sheets
2025-08-21 19:03:02,257 - INFO - Payload preview: [['688f97f9af6fe3b64062e328', 'kfekf', 97, True, 'social_media', '', '', 'ss', 60.0, 170000.0, 43.0, 757583.41, 68856588.33, 225872.08, '2025-08-03T17:10:17.787Z', '2025-08-03T17:10:25.149Z', '', '', ''], ['68904932af6fe3b64062e33f', 'Nishant Neeraj', 9871077114, True, 'social_media', '', '', 'Delhi', 50.0, 100000.0, 65.0, 2875869.83, 37906250.53, 2160056.0, '2025-08-04T05:46:26.043Z', '2025-08-04T05:46:44.093Z', '', '', '']]
2025-08-21 19:03:33,169 - ERROR - ❌ Failed to upload data: HTTPSConnectionPool(host='script.google.com', port=443): Read timed out. (read timeout=30)


💥 Upload failed! Check the logs above for details. False


In [44]:
BATCH_SIZE = 500

def chunked_upload(client, data, batch_size=BATCH_SIZE):
    total = len(data)
    for start in range(0, total, batch_size):
        end = start + batch_size
        batch = data[start:end]
        print(f"📤 Uploading rows {start}–{end-1} ({len(batch)} rows)")
        
        success = client.upload_data(batch)
        if not success:
            print(f"❌ Failed to upload batch {start}–{end-1}")
            break


# processed_data = df.values.tolist()  # already prepared
chunked_upload(sheets_client, processed_data, batch_size=500)


2025-08-21 20:00:01,821 - INFO - Uploading 500 rows to Google Sheets
2025-08-21 20:00:01,821 - INFO - Payload preview: [['689565345b559c022b76aab8', 'Nishkam', 9897320666, True, 'pnb', 8038685.0, '', '', 35.0, 35000.0, 60.0, 1802585.7, 41956930.52, 1125462.76, '2025-08-08T02:47:16.633Z', '2025-08-08T02:47:38.988Z', '', '', ''], ['689565cb17c94b36ba15df84', 'Sumit Kumar', 7037531182, True, 'pnb', 8036334.0, '', '', 28.0, 50000.0, 60.0, 3872032.01, 90125300.44, 2417542.66, '2025-08-08T02:49:47.984Z', '2025-08-08T02:50:00.141Z', '', '', '']]


📤 Uploading rows 0–499 (500 rows)


2025-08-21 20:00:32,116 - ERROR - ❌ Failed to upload data: HTTPSConnectionPool(host='script.google.com', port=443): Read timed out. (read timeout=30)


❌ Failed to upload batch 0–499


## Usage Instructions

1. **Update Configuration**: Modify the `CSV_FILE_PATH` variable in the configuration cell to point to your CSV file
2. **Run All Cells**: Execute all cells in order (Shift + Enter or Run All)
3. **Check Results**: Monitor the output and logs for success/failure status

## Expected CSV Format

Your CSV should have columns matching your User model structure:
- `_id`: User ID
- `name`: User name
- `mobile`: Mobile number
- `consent`: Consent status
- `channel`: Channel (agency, pnb, jkb, kbl, psf, social_media)
- `employeeCode`: Employee code
- `buisnessCode`: Business code
- `city`: City
- `age`: Age
- `monthlyExpense`: Monthly expense
- `retirementAge`: Retirement age
- `futureValue`: Future value
- `retirementCorpus`: Retirement corpus
- `yearlyInvestment`: Yearly investment
- `createdAt`: Creation timestamp (ISO format)
- `updatedAt`: Update timestamp (ISO format)
- `utm`: UTM parameter
- `utm_source`: UTM source
- `utm_campaign`: UTM campaign

## Notes

- Missing columns will be filled with empty strings
- Dates are automatically formatted to DD-MM-YYYY HH:MM:SS format
- The script handles null/empty values gracefully
- All data is uploaded in append mode to your existing sheet

In [ ]:
# Optional: Create a sample CSV file for testing
def create_sample_csv():
    """Create a sample CSV file for testing purposes"""
    sample_data = {
        '_id': ['507f1f77bcf86cd799439011', '507f1f77bcf86cd799439012', '507f1f77bcf86cd799439013'],
        'name': ['John Doe', 'Jane Smith', 'Bob Johnson'],
        'mobile': [9876543210, 9876543211, 9876543212],
        'consent': [True, True, False],
        'channel': ['agency', 'pnb', 'social_media'],
        'employeeCode': [1234567, 1234568, None],
        'buisnessCode': [87654321, 87654322, None],
        'city': ['Mumbai', 'Delhi', 'Bangalore'],
        'age': [30, 28, 35],
        'monthlyExpense': [50000, 45000, 60000],
        'retirementAge': [60, 58, 65],
        'futureValue': [1000000, 800000, 1200000],
        'retirementCorpus': [5000000, 4000000, 6000000],
        'yearlyInvestment': [100000, 90000, 120000],
        'createdAt': ['2024-01-15T10:30:00Z', '2024-01-15T11:00:00Z', '2024-01-15T11:30:00Z'],
        'updatedAt': ['2024-01-15T10:30:00Z', '2024-01-15T11:00:00Z', '2024-01-15T11:30:00Z'],
        'utm': ['google', 'facebook', 'instagram'],
        'utm_source': ['google_ads', 'facebook_ads', 'instagram_ads'],
        'utm_campaign': ['Fire Campaign', 'Fire Campaign', 'Fire Campaign']
    }
    
    df = pd.DataFrame(sample_data)
    df.to_csv('sample_data.csv', index=False)
    print("✅ Sample CSV file 'sample_data.csv' created successfully!")
    print("You can now use this file for testing by setting CSV_FILE_PATH = 'sample_data.csv'")
    
    return df

# Uncomment the line below to create a sample CSV file
# sample_df = create_sample_csv()

In [ ]:
# Add the missing helper functions and fix the process_dataframe function\n
def format_date(iso_date):\n
    """Transform ISO date to DD-MM-YYYY HH:MM:SS format"""\n
    if pd.isna(iso_date) or iso_date == "null" or iso_date == "":\n
        return ""\n
    \n
    try:\n
        if isinstance(iso_date, str):\n
            # Handle ISO string format\n
            date_obj = pd.to_datetime(iso_date)\n
        else:\n
            date_obj = pd.to_datetime(iso_date)\n
        \n
        return date_obj.strftime("%d-%m-%Y %H:%M:%S")\n
    except:\n
        logger.warning(f"Could not parse date: {iso_date}")\n
        return ""\n
\n
def clean_value(value):\n
    """Clean and format values for sheets"""\n
    if pd.isna(value) or value == "null" or value == "":\n
        return ""\n
    return str(value)\n
\n
# Fix the process_dataframe function\n
def process_dataframe(df: pd.DataFrame) -> List[List[str]]:\n
    """Process DataFrame and return formatted values for sheets"""\n
    # Define the expected column order based on your User model\n
    expected_columns = [\n
        '_id', 'name', 'mobile', 'consent', 'channel', 'employeeCode', \n
        'buisnessCode', 'city', 'age', 'monthlyExpense', 'retirementAge', \n
        'futureValue', 'retirementCorpus', 'yearlyInvestment', 'createdAt', \n
        'updatedAt', 'utm', 'utm_source', 'utm_campaign'\n
    ]\n
    \n
    # Create a new DataFrame with the expected structure\n
    processed_df = pd.DataFrame()\n
    \n
    for col in expected_columns:\n
        if col in df.columns:\n
            if col in ['createdAt', 'updatedAt']:\n
                processed_df[col] = df[col].apply(format_date)\n
            else:\n
                processed_df[col] = df[col].apply(clean_value)\n
        else:\n
            # Add missing columns with empty values\n
            processed_df[col] = ""\n
    \n
    # Convert to list of lists for sheets\n
    values = processed_df.values.tolist()\n
    \n
    return values\n
\n
print("✅ Helper functions added successfully!")\n
print("Now you can run the data processing cell again.")

In [ ]:
# Fix the GoogleSheetsClient class\n
class GoogleSheetsClient:\n
    """Client for interacting with Google Sheets via Apps Script endpoint"""\n
    \n
    def __init__(self, sheet_url: str, sheet_id: str):\n
        self.sheet_url = sheet_url\n
        self.sheet_id = sheet_id\n
        self.session = requests.Session()\n
        self.session.headers.update({\n
            'Content-Type': 'application/json',\n
            'User-Agent': 'CSV-to-Sheets-Uploader/1.0'\n
        })\n
    \n
    def upload_data(self, data: List[List[str]]) -> bool:\n
        """Upload data to Google Sheets"""\n
        try:\n
            payload = {\n
                "mode": "append",\n
                "data": data,\n
                "sheetId": self.sheet_id\n
            }\n
            \n
            logger.info(f"Uploading {len(data)} rows to Google Sheets")\n
            logger.info(f"Payload preview: {payload['data'][:2] if len(payload['data']) > 0 else 'No data'}")\n
            \n
            response = self.session.post(\n
                self.sheet_url,\n
                json=payload,\n
                timeout=30\n
            )\n
            \n
            response.raise_for_status()\n
            \n
            logger.info("✅ Data successfully uploaded to Google Sheets")\n
            logger.info(f"Response: {response.text}")\n
            return True\n
            \n
        except requests.exceptions.RequestException as e:\n
            logger.error(f"❌ Failed to upload data: {e}")\n
            return False\n
        except Exception as e:\n
            logger.error(f"❌ Unexpected error during upload: {e}")\n
            return False\n
\n
# Reinitialize the client with the fixed class\n
sheets_client = GoogleSheetsClient(SHEET_URL, SHEET_ID)\n
print("✅ GoogleSheetsClient fixed and reinitialized!")